In [1]:
%cd ../..

c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1


In [2]:
import numpy as np
import pandas as pd
import time
import plotly.express as px
import plotly.graph_objects as go
import os
import plotly.io as pio
pio.templates.default = "plotly_white"

from pathlib import Path
from tqdm.autonotebook import tqdm
import warnings
import humanize
import joblib

from sklearn.preprocessing import StandardScaler

from src.utils.ts_utils import forecast_bias, metrics_adapter, mase, mse, mae
from src.utils.general import LogTime
from src.utils import plotting_utils
from src.forecasting.ml_forecasting import MissingValueConfig, FeatureConfig, ModelConfig, MLForecast, calculate_metrics
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

C:\Users\tacke\AppData\Local\Temp\ipykernel_43076\1168638155.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
os.makedirs("imgs/chapter_8", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")
output = Path("data/london_smart_meters/output")

In [4]:
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", title=""):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=900,
            height=500,
            title_text=title,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=12),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=12),
            )
        )
    return fig

In [5]:
def highlight_abs_min(s, props=''):
    return np.where(s == np.nanmin(np.abs(s.values)), props, '')

In [6]:
try:
    #Readin the missing value imputed and train test split data
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    auto_stat_target = pd.read_parquet(preprocessed/"selected_blocks_train_auto_stat_target.parquet")
    transformer_pipelines = joblib.load(preprocessed/"auto_transformer_pipelines_train.pkl")
    #Reading in validation as test
    test_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")
    # Joining the transformed target 
    train_df = train_df.set_index(['LCLid','timestamp']).join(auto_stat_target).reset_index()
    # #Renaming energy
    # test_df.rename(columns={"energy_consumption":"energy_consumption_auto_stat"}, inplace=True)
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb and 02-Dealing with Non-Stationarity.ipynb in Chapter06 and Chapter07
    </div>
    """))

In [7]:
len(train_df.LCLid.unique())

150

In [8]:
try:
    baseline_metrics_df = pd.read_pickle(output/"ml_single_step_metrics_val_df.pkl")
    baseline_aggregate_metrics_df = pd.read_pickle(output/"ml_single_step_aggregate_metrics_val.pkl")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Forecasting with ML.ipynb in Chapter08
    </div>
    """))

In [9]:
train_df.head()

,LCLid,timestamp,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,timestamp_Minute_sin_2,timestamp_Minute_sin_3,timestamp_Minute_sin_4,timestamp_Minute_sin_5,timestamp_Minute_cos_1,timestamp_Minute_cos_2,timestamp_Minute_cos_3,timestamp_Minute_cos_4,timestamp_Minute_cos_5,energy_consumption_auto_stat
0,MAC000061,2012-01-01 00:00:00,0.114,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0,0.555465
1,MAC000061,2012-01-01 00:30:00,0.113,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,-1.133108e-15,2.143751e-15,-2.266215e-15,6.123234e-16,-1.0,1.0,-1.0,1.0,-1.0,0.560217
2,MAC000061,2012-01-01 01:00:00,0.113,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0,0.563023
3,MAC000061,2012-01-01 01:30:00,0.098,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,-1.133108e-15,2.143751e-15,-2.266215e-15,6.123234e-16,-1.0,1.0,-1.0,1.0,-1.0,0.560639
4,MAC000061,2012-01-01 02:00:00,0.060,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0,0.553762


# Feature Definition

In [10]:
feat_config = FeatureConfig(
    date="timestamp",
    target="energy_consumption_auto_stat",
    original_target="energy_consumption",
    continuous_features=[
        "visibility",
        "windBearing",
        "temperature",
        "dewPoint",
        "pressure",
        "apparentTemperature",
        "windSpeed",
        "humidity",
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
        "energy_consumption_rolling_3_mean",
        "energy_consumption_rolling_3_std",
        "energy_consumption_rolling_6_mean",
        "energy_consumption_rolling_6_std",
        "energy_consumption_rolling_12_mean",
        "energy_consumption_rolling_12_std",
        "energy_consumption_rolling_48_mean",
        "energy_consumption_rolling_48_std",
        "energy_consumption_48_seasonal_rolling_3_mean",
        "energy_consumption_48_seasonal_rolling_3_std",
        "energy_consumption_336_seasonal_rolling_3_mean",
        "energy_consumption_336_seasonal_rolling_3_std",
        "energy_consumption_ewma_span_2880",
        "energy_consumption_ewma_span_336",
        "energy_consumption_ewma_span_48",
        "timestamp_Elapsed",
        "timestamp_Month_sin_1",
        "timestamp_Month_sin_2",
        "timestamp_Month_sin_3",
        "timestamp_Month_sin_4",
        "timestamp_Month_sin_5",
        "timestamp_Month_cos_1",
        "timestamp_Month_cos_2",
        "timestamp_Month_cos_3",
        "timestamp_Month_cos_4",
        "timestamp_Month_cos_5",
        "timestamp_Hour_sin_1",
        "timestamp_Hour_sin_2",
        "timestamp_Hour_sin_3",
        "timestamp_Hour_sin_4",
        "timestamp_Hour_sin_5",
        "timestamp_Hour_cos_1",
        "timestamp_Hour_cos_2",
        "timestamp_Hour_cos_3",
        "timestamp_Hour_cos_4",
        "timestamp_Hour_cos_5",
        "timestamp_Minute_sin_1",
        "timestamp_Minute_sin_2",
        "timestamp_Minute_sin_3",
        "timestamp_Minute_sin_4",
        "timestamp_Minute_sin_5",
        "timestamp_Minute_cos_1",
        "timestamp_Minute_cos_2",
        "timestamp_Minute_cos_3",
        "timestamp_Minute_cos_4",
        "timestamp_Minute_cos_5",
    ],
    categorical_features=[
        "holidays",
        "precipType",
        "icon",
        "summary",
        "timestamp_Month",
        "timestamp_Quarter",
        "timestamp_WeekDay",
        "timestamp_Dayofweek",
        "timestamp_Dayofyear",
        "timestamp_Hour",
        "timestamp_Minute",
    ],
    boolean_features=[
        "timestamp_Is_quarter_end",
        "timestamp_Is_quarter_start",
        "timestamp_Is_year_end",
        "timestamp_Is_year_start",
        "timestamp_Is_month_start",
    ],
    index_cols=["timestamp"],
    exogenous_features=[
        "holidays",
        "precipType",
        "icon",
        "summary",
        "visibility",
        "windBearing",
        "temperature",
        "dewPoint",
        "pressure",
        "apparentTemperature",
        "windSpeed",
        "humidity",
    ],
)

# Missing Value Handling

In [11]:
missing_value_config = MissingValueConfig(
    bfill_columns=[
        "energy_consumption_lag_1",
        "energy_consumption_lag_2",
        "energy_consumption_lag_3",
        "energy_consumption_lag_4",
        "energy_consumption_lag_5",
        "energy_consumption_lag_46",
        "energy_consumption_lag_47",
        "energy_consumption_lag_48",
        "energy_consumption_lag_49",
        "energy_consumption_lag_50",
        "energy_consumption_lag_334",
        "energy_consumption_lag_335",
        "energy_consumption_lag_336",
        "energy_consumption_lag_337",
        "energy_consumption_lag_338",
        "energy_consumption_rolling_3_mean",
        "energy_consumption_rolling_3_std",
        "energy_consumption_rolling_6_mean",
        "energy_consumption_rolling_6_std",
        "energy_consumption_rolling_12_mean",
        "energy_consumption_rolling_12_std",
        "energy_consumption_rolling_48_mean",
        "energy_consumption_rolling_48_std",
        "energy_consumption_48_seasonal_rolling_3_mean",
        "energy_consumption_48_seasonal_rolling_3_std",
        "energy_consumption_336_seasonal_rolling_3_mean",
        "energy_consumption_336_seasonal_rolling_3_std",
        "energy_consumption_ewma__span_2880",
        "energy_consumption_ewma__span_336",
        "energy_consumption_ewma__span_48",
    ],
    ffill_columns=[],
    zero_fill_columns=[],
)

# Running ML Forecast for all consumers

Running Lasso Regression, XGB Random Forest, and LightGBM

In [12]:
def evaluate_model(model_config, feature_config, missing_config, target_transformer, train_features, train_target, test_features, test_target, train_target_original=None):
    ml_model = MLForecast(model_config=model_config, feature_config=feat_config, missing_config=missing_value_config, target_transformer=target_transformer)
    ml_model.fit(train_features, train_target, is_transformed=True)
    y_pred = ml_model.predict(test_features)
    feat_df = ml_model.feature_importance()
    metrics = calculate_metrics(test_target, y_pred, model_config.name, train_target_original)
    return y_pred, metrics, feat_df

In [13]:
from sklearn.linear_model import LassoCV
from xgboost import XGBRFRegressor
from lightgbm import LGBMRegressor

In [14]:
lcl_ids = sorted(train_df.LCLid.unique())
models_to_run = [
    ModelConfig(model = LassoCV(), name="Lasso Regression", normalize=True, fill_missing=True),
    ModelConfig(model = XGBRFRegressor(random_state=42, max_depth=4), name="XGB Random Forest", normalize=False, fill_missing=False),
    ModelConfig(model = LGBMRegressor(random_state=42), name="LightGBM", normalize=False, fill_missing=False)
]

In [15]:
all_preds = []
all_metrics = []
#We can parallelize this loop to run this faster
for lcl_id in tqdm(lcl_ids):
    for model_config in models_to_run:
        model_config = model_config.clone()
        X_train, y_train, y_train_orig = feat_config.get_X_y(train_df.loc[train_df.LCLid==lcl_id,:], categorical=False, exogenous=False)
        X_test, _, y_test_orig = feat_config.get_X_y(test_df.loc[test_df.LCLid==lcl_id,:], categorical=False, exogenous=False)
        transformer = transformer_pipelines[lcl_id]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            y_pred, metrics, feat_df = evaluate_model(model_config, feat_config, missing_value_config, transformer, X_train, y_train, X_test, y_test_orig, y_train_orig)
        y_pred.name = "predictions"
        y_pred = y_pred.to_frame()
        y_pred['LCLid'] = lcl_id
        y_pred['Algorithm'] = model_config.name + "_auto_stat"
        metrics["LCLid"] = lcl_id
        metrics["Algorithm"] = model_config.name + "_auto_stat"
        y_pred['energy_consumption'] = y_test_orig.values
        all_preds.append(y_pred)
        all_metrics.append(metrics)

  0%|          | 0/150 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018910 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8115
[LightGBM] [Info] Number of data points in the train set: 35088, number of used features: 59
[LightGBM] [Info] Start training from score 0.549685
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.041632 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8115
[LightGBM] [Info] Number of data points in the train set: 35088, number of used features: 59
[LightGBM] [Info] Start training from score 0.573847
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011831 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8115
[LightGBM] [Info] Number of data points in the train set: 35088, number of used features: 59
[LightGBM] [Info] Start tra

In [16]:
pred_df = pd.concat(all_preds)
pred_df.head()

,predictions,LCLid,Algorithm,energy_consumption
timestamp,,,,
2014-01-01 00:00:00,0.119527,MAC000061,Lasso Regression_auto_stat,0.165
2014-01-01 00:30:00,0.105027,MAC000061,Lasso Regression_auto_stat,0.167
2014-01-01 01:00:00,0.129575,MAC000061,Lasso Regression_auto_stat,0.150
2014-01-01 01:30:00,0.120934,MAC000061,Lasso Regression_auto_stat,0.091
2014-01-01 02:00:00,0.080307,MAC000061,Lasso Regression_auto_stat,0.047


In [17]:
metrics_df = pd.DataFrame(all_metrics)
metrics_df.head()

,Algorithm,MAE,MSE,MASE,Forecast Bias,LCLid
0,Lasso Regression_auto_stat,0.036641,0.002990,1.048579,-5.462709,MAC000061
1,XGB Random Forest_auto_stat,0.035695,0.002971,1.021503,0.232144,MAC000061
2,LightGBM_auto_stat,0.032544,0.002720,0.931330,-0.832978,MAC000061
3,Lasso Regression_auto_stat,0.068971,0.027052,0.916061,1.826489,MAC000062
4,XGB Random Forest_auto_stat,0.071279,0.027271,0.946720,1.331908,MAC000062


# Evaluation of ML Forecast

In [18]:
from src.utils import ts_utils

In [19]:
baseline_aggregate_metrics_df

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,Naive,0.088162,0.044981,1.089973,-0.002962
1,Seasonal Naive,0.129197,0.077654,1.581947,-1.001533
2,Lasso Regression,0.080147,0.027100,1.005135,-0.295070
3,XGB Random Forest,0.080650,0.030574,1.016154,-2.476618
4,LightGBM,0.077107,0.027468,0.977439,0.032495


In [20]:
metrics = baseline_aggregate_metrics_df.to_dict(orient="records")

In [21]:

for model in models_to_run:
    pred_mask = pred_df.Algorithm==model.name+"_auto_stat"
    metric_mask = metrics_df.Algorithm==model.name+"_auto_stat"
    metrics.append({
    "Algorithm": model.name+"_auto_stat",
    "MAE": ts_utils.mae(pred_df.loc[pred_mask,"energy_consumption"], pred_df.loc[pred_mask,"predictions"]),
    "MSE": ts_utils.mse(pred_df.loc[pred_mask,"energy_consumption"], pred_df.loc[pred_mask,"predictions"]),
    "meanMASE": metrics_df.loc[metric_mask, "MASE"].mean(),
    "Forecast Bias": ts_utils.forecast_bias_aggregate(pred_df.loc[pred_mask,"energy_consumption"], pred_df.loc[pred_mask,"predictions"])
})

In [22]:
agg_metrics_df = pd.DataFrame(metrics)
agg_metrics_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]).apply(highlight_abs_min, props='color:black;background-color:lightgreen', axis=0, subset=['Forecast Bias'])

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,Naive,0.088,0.045,1.090,-0.00%
1,Seasonal Naive,0.129,0.078,1.582,-1.00%
2,Lasso Regression,0.080,0.027,1.005,-0.30%
3,XGB Random Forest,0.081,0.031,1.016,-2.48%
4,LightGBM,0.077,0.027,0.977,0.03%
5,Lasso Regression_auto_stat,0.083,0.030,1.055,-3.50%
6,XGB Random Forest_auto_stat,0.086,0.033,1.097,-8.34%
7,LightGBM_auto_stat,0.079,0.029,1.005,-4.36%


In [23]:
fig = px.histogram(metrics_df, 
                   x="MASE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,2.5])
# fig.write_image("imgs/chapter_7/mase_dist.png")
fig.show()

In [24]:
fig = px.histogram(metrics_df, 
                   x="MAE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=100, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MAE", ylabel="Probability Density", title="Distribution of MAE in the dataset")
# fig.write_image("imgs/chapter_7/mae_dist.png")
fig.update_layout(xaxis_range=[0,0.4])
fig.show()

In [25]:
fig = px.histogram(metrics_df, 
                   x="MSE", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MSE", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,0.3])
# fig.write_image("imgs/chapter_7/mse_dist.png")
fig.show()

In [26]:
fig = px.histogram(metrics_df, 
                   x="Forecast Bias", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=250,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="Forecast Bias", ylabel="Probability Density", title="Distribution of Forecast Bias in the dataset")
fig.update_layout(xaxis_range=[-50,30])
# fig.write_image("imgs/chapter_7/bias_dist.png")
fig.show()

# Saving the Baseline Forecasts and Metrics

In [27]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [28]:
pred_df.to_pickle(output/"ml_single_step_prediction_auto_stationary_val_df.pkl")
metrics_df.to_pickle(output/"ml_single_step_metrics_auto_stationary_val_df.pkl")
agg_metrics_df.to_pickle(output/"ml_single_step_aggregate_metrics_auto_stationary_val.pkl")